# 인구소멸 위기지역- 전처리(Part1)
* 인구소멸위기 지역
* 인구소멸위험지수: 65세 이상 고령인구 수 대비 20~39세 여성인구 수로 계산
  * 지수가 0.5 미만이면 인구소멸 위험 지역, 0.2 미만이면 인구소멸 고위험 지역으로 간주

In [1]:
import numpy as np
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/mnt')

Mounted at /content/mnt


In [3]:
data_folder = '/content/mnt/MyDrive/Data/인구 소멸위기지역'

### 데이터 불러오기

In [23]:
df = pd.read_csv(f'{data_folder}/시군구_성_연령_5세_별_인구2_2020.csv', encoding='euc-kr')
print(df.shape)
df.head(260).tail()

(837, 28)


,광역시도,시군구,행정구,구분,인구수,0 - 4세,5 - 9세,10 - 14세,15 - 19세,20 - 24세,25 - 29세,30 - 34세,35 - 39세,40 - 44세,45 - 49세,50 - 54세,55 - 59세,60 - 64세,65 - 69세,70 - 74세,75 - 79세,80세 이상,80 - 84세,85세 이상,85 - 89세,90 - 94세,95 - 99세,100세 이상
255,경기도,수원시,소계,계,1181470,43910,56741,57093,63808,86365,99616,84580,96057,96583,105839,102027,92014,70689,43315,31267,23774,27796,16028,11768,8044,2957,671,96
256,경기도,수원시,소계,남자,594380,22626,29290,29411,33178,44785,53229,44517,49009,48829,53013,50062,46571,35430,20789,14491,10013,9142,5900,3242,2414,678,137,14
257,경기도,수원시,소계,여자,587090,21284,27451,27682,30630,41581,46387,40063,47048,47755,52826,51965,45444,35259,22527,16776,13761,18654,10129,8526,5630,2280,535,82
258,경기도,수원시,장안구,계,273908,7659,11548,13483,16100,22511,21523,15325,17922,20514,25397,25831,22953,17975,11311,8549,6870,8443,4806,3637,2459,944,207,28
259,경기도,수원시,장안구,남자,137852,3904,6034,6954,8498,12166,11922,8235,9021,10143,12591,12690,11637,9000,5418,3905,2892,2846,1819,1027,755,225,46,2


In [24]:
df['시군구'][df.행정구.notnull()] = df.행정구[df.행정구.notnull()]
df.head(260).tail()

,광역시도,시군구,행정구,구분,인구수,0 - 4세,5 - 9세,10 - 14세,15 - 19세,20 - 24세,25 - 29세,30 - 34세,35 - 39세,40 - 44세,45 - 49세,50 - 54세,55 - 59세,60 - 64세,65 - 69세,70 - 74세,75 - 79세,80세 이상,80 - 84세,85세 이상,85 - 89세,90 - 94세,95 - 99세,100세 이상
255,경기도,소계,소계,계,1181470,43910,56741,57093,63808,86365,99616,84580,96057,96583,105839,102027,92014,70689,43315,31267,23774,27796,16028,11768,8044,2957,671,96
256,경기도,소계,소계,남자,594380,22626,29290,29411,33178,44785,53229,44517,49009,48829,53013,50062,46571,35430,20789,14491,10013,9142,5900,3242,2414,678,137,14
257,경기도,소계,소계,여자,587090,21284,27451,27682,30630,41581,46387,40063,47048,47755,52826,51965,45444,35259,22527,16776,13761,18654,10129,8526,5630,2280,535,82
258,경기도,장안구,장안구,계,273908,7659,11548,13483,16100,22511,21523,15325,17922,20514,25397,25831,22953,17975,11311,8549,6870,8443,4806,3637,2459,944,207,28
259,경기도,장안구,장안구,남자,137852,3904,6034,6954,8498,12166,11922,8235,9021,10143,12591,12690,11637,9000,5418,3905,2892,2846,1819,1027,755,225,46,2


In [25]:
# 소계 데이터 제외
df = df[df.시군구 != '소계']
print(df.shape)
df.head()

(750, 28)


,광역시도,시군구,행정구,구분,인구수,0 - 4세,5 - 9세,10 - 14세,15 - 19세,20 - 24세,25 - 29세,30 - 34세,35 - 39세,40 - 44세,45 - 49세,50 - 54세,55 - 59세,60 - 64세,65 - 69세,70 - 74세,75 - 79세,80세 이상,80 - 84세,85세 이상,85 - 89세,90 - 94세,95 - 99세,100세 이상
6,서울특별시,종로구,NaN,계,144866,3309,4549,5120,6182,10602,12668,9995,9796,9650,12221,12287,12160,10735,7637,6245,5450,6265,3682,2583,1749,639,169,26
7,서울특별시,종로구,NaN,남자,70613,1668,2297,2575,3099,4976,6526,5005,4820,4636,5993,6072,6180,5409,3744,2842,2369,2406,1517,889,642,189,52,6
8,서울특별시,종로구,NaN,여자,74253,1641,2252,2545,3083,5627,6142,4991,4976,5014,6228,6215,5981,5326,3893,3403,3081,3859,2165,1694,1107,451,117,20
9,서울특별시,중구,NaN,계,121520,3483,3611,3093,3869,7605,10768,10031,10051,8465,9184,9864,10108,9512,6872,5475,4501,5030,2904,2126,1429,558,124,16
10,서울특별시,중구,NaN,남자,59536,1772,1852,1550,1976,3543,5289,4981,5051,4376,4636,5005,5127,4782,3380,2523,1943,1754,1125,629,439,153,32,6


In [27]:
df['20-39세'] = df['20 - 24세'] + df['25 - 29세'] + df['30 - 34세'] + df['35 - 39세']
df['65세이상'] = df['65 - 69세'] + df['70 - 74세'] + df['75 - 79세'] + df['80세 이상']
df = df[['광역시도', '시군구', '구분', '인구수', '20-39세', '65세이상']]
df.head()

,광역시도,시군구,구분,인구수,20-39세,65세이상
6,서울특별시,종로구,계,144866,43061,25597
7,서울특별시,종로구,남자,70613,21327,11361
8,서울특별시,종로구,여자,74253,21736,14236
9,서울특별시,중구,계,121520,38455,21878
10,서울특별시,중구,남자,59536,18864,9600


In [28]:
# pivot_table을 만들어서 시군구 단위로 grouping

pop = pd.pivot_table(df, 
                     index=['광역시도', '시군구'], columns=['구분'], 
                     values=['인구수', '20-39세', '65세이상'])
print(pop.shape)
pop.head()

(250, 9)


20-39세                65세이상                   인구수                
구분             계     남자     여자      계     남자     여자       계      남자      여자
광역시도 시군구                                                                   
강원도   강릉시  47006  25199  21807  43830  18604  25227  211643  105025  106618
      고성군   5217   3175   2042   7755   3237   4519   26792   13864   12928
      동해시  18727  10469   8258  17572   7624   9949   89814   45572   44242
      삼척시  13583   7469   6117  16077   6766   9311   65623   33275   32348
      속초시  18080   9909   8171  15170   6345   8826   81497   40312   41185

In [29]:
# 인구 소멸비율 계산 
pop['소멸비율'] = pop['20-39세', '여자'] / pop['65세이상', '계']
pop.head()

20-39세                65세이상  ...     인구수                      소멸비율
구분             계     남자     여자      계  ...       계      남자      여자          
광역시도 시군구                               ...                                  
강원도   강릉시  47006  25199  21807  43830  ...  211643  105025  106618  0.497536
      고성군   5217   3175   2042   7755  ...   26792   13864   12928  0.263314
      동해시  18727  10469   8258  17572  ...   89814   45572   44242  0.469952
      삼척시  13583   7469   6117  16077  ...   65623   33275   32348  0.380481
      속초시  18080   9909   8171  15170  ...   81497   40312   41185  0.538629

[5 rows x 10 columns]

In [30]:
pop['소멸위기지역'] = pop['소멸비율'] < 0.5 
pop['소멸위기고위험지역'] = pop['소멸비율'] < 0.2
pop.head()

20-39세                65세이상  ...     인구수      소멸비율 소멸위기지역 소멸위기고위험지역
구분             계     남자     여자      계  ...      여자                           
광역시도 시군구                               ...                                   
강원도   강릉시  47006  25199  21807  43830  ...  106618  0.497536   True     False
      고성군   5217   3175   2042   7755  ...   12928  0.263314   True     False
      동해시  18727  10469   8258  17572  ...   44242  0.469952   True     False
      삼척시  13583   7469   6117  16077  ...   32348  0.380481   True     False
      속초시  18080   9909   8171  15170  ...   41185  0.538629  False     False

[5 rows x 12 columns]

In [31]:
# 인구 소멸 위기지역
# 두번째 인덱스를 가져와라 
crisis_region = pop[pop.소멸위기지역].index.get_level_values(1)

print(len(crisis_region))
print(crisis_region)

102
Index([' 강릉시', ' 고성군', ' 동해시', ' 삼척시', ' 양구군', ' 양양군', ' 영월군', ' 인제군', ' 정선군',
       ' 철원군',
       ...
       ' 청양군', ' 태안군', ' 홍성군', ' 괴산군', ' 단양군', ' 보은군', ' 영동군', ' 옥천군', ' 음성군',
       ' 제천시'],
      dtype='object', name='시군구', length=102)


In [33]:
# 인구 소멸 고위험지역
high_crisis_region = pop[pop.소멸위기고위험지역].index.get_level_values(1)

print(len(high_crisis_region))
print(high_crisis_region)

22
Index([' 남해군', ' 산청군', ' 의령군', ' 하동군', ' 합천군', ' 군위군', ' 봉화군', ' 영덕군', ' 영양군',
       ' 의성군', ' 청도군', ' 청송군', ' 고흥군', ' 곡성군', ' 보성군', ' 신안군', ' 함평군', ' 임실군',
       ' 부여군', ' 서천군', ' 청양군', ' 보은군'],
      dtype='object', name='시군구')


In [34]:
pop.columns.get_level_values(0)

Index(['20-39세', '20-39세', '20-39세', '65세이상', '65세이상', '65세이상', '인구수', '인구수',
       '인구수', '소멸비율', '소멸위기지역', '소멸위기고위험지역'],
      dtype='object')

In [36]:
for i in range(len(pop.columns.get_level_values(0))):
  print(pop.columns.get_level_values(0)[i]+ pop.columns.get_level_values(1)[i])

20-39세계
20-39세남자
20-39세여자
65세이상계
65세이상남자
65세이상여자
인구수계
인구수남자
인구수여자
소멸비율
소멸위기지역
소멸위기고위험지역


In [37]:
tmp_col = [pop.columns.get_level_values(0)[i]+ pop.columns.get_level_values(1)[i]
           for i in range(len(pop.columns.get_level_values(0)))]
pop.columns = tmp_col
pop.reset_index(inplace=True)
pop['시군구'] = pop.시군구.apply(lambda x: x.strip())       # 앞에 빈 칸을 뺀다
pop.head()

,광역시도,시군구,20-39세계,20-39세남자,20-39세여자,65세이상계,65세이상남자,65세이상여자,인구수계,인구수남자,인구수여자,소멸비율,소멸위기지역,소멸위기고위험지역
0,강원도,강릉시,47006,25199,21807,43830,18604,25227,211643,105025,106618,0.497536,True,False
1,강원도,고성군,5217,3175,2042,7755,3237,4519,26792,13864,12928,0.263314,True,False
2,강원도,동해시,18727,10469,8258,17572,7624,9949,89814,45572,44242,0.469952,True,False
3,강원도,삼척시,13583,7469,6117,16077,6766,9311,65623,33275,32348,0.380481,True,False
4,강원도,속초시,18080,9909,8171,15170,6345,8826,81497,40312,41185,0.538629,False,False


In [39]:
pop.to_csv(f'{data_folder}/시군구_전처리완료.csv', encoding='euc-kr', index=False)